In [ ]:

%cd /content


!git clone https://github.com/adikan2k/Final-Project-Group-LexiCore


%cd Final-Project-Group-LexiCore


!ls

/content
Cloning into 'Final-Project-Group-LexiCore'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 277 (delta 109), reused 239 (delta 95), pack-reused 0 (from 0)
Receiving objects: 100% (277/277), 23.47 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/Final-Project-Group-LexiCore
 code			    'Final Group Project Report'   README.md
'Final Group Presentation'   LICENSE


In [ ]:
!pip install spacy langdetect pyarrow pandas nltk


!python -m spacy download en_core_web_sm

import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
!ls src/preprocessing

__init__.py  preprocess.py  __pycache__


In [ ]:
import spacy
from langdetect import detect, LangDetectException
import unicodedata
import time

nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])
nlp.max_length = 2_000_000  

def get_raw_text(row, full_col="full_text", abstract_col="abstract"):
    """
    Prefer full text if available, otherwise abstract.
    Change column names here if your dataset uses different ones.
    """
    full = row.get(full_col)
    abstract = row.get(abstract_col)
    if isinstance(full, str) and full.strip():
        return full
    if isinstance(abstract, str) and abstract.strip():
        return abstract
    return ""


def normalize_unicode(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return unicodedata.normalize("NFKC", text)


def is_english(text: str, min_chars: int = 200) -> bool:
    """
    Detect if text is English using langdetect on a small sample.
    """
    if not isinstance(text, str) or not text.strip():
        return False
    sample = text[: max(min_chars, 50)]
    try:
        return detect(sample) == "en"
    except LangDetectException:
        return False


def get_sentences(text: str):
    """
    Split text into sentences using spaCy, for summarization later.
    """
    if not text:
        return []
    doc = nlp(text)
    return [s.text.strip() for s in doc.sents if s.text.strip()]


def run_full_preprocessing_pipeline(
    df: pd.DataFrame,
    full_col: str = "full_text",
    abstract_col: str = "abstract",
) -> pd.DataFrame:
    """
    Uses your teammate's preprocess_text() plus extra steps:

    - build raw_text from full + abstract
    - unicode normalization
    - regex cleaning (clean_text + remove_special_chars)
    - language detection (drop non-English)
    - sentence splitting
    - tokenization/lemmatization via preprocess_text
    - num_tokens (benchmark)
    """
    df = df.copy()

 
    print("Step 1: create raw_text from full + abstract...")
    df["raw_text"] = df.apply(
        lambda row: get_raw_text(row, full_col=full_col, abstract_col=abstract_col),
        axis=1,
    )

    print("Step 2: unicode normalization + regex cleaning...")
    df["raw_text_clean"] = (
        df["raw_text"]
        .astype(str)
        .apply(normalize_unicode)
        .apply(clean_text)          
        .apply(remove_special_chars)  
    )

   
    print("Step 3: language detection (English only)...")
    df["is_english"] = df["raw_text_clean"].apply(is_english)
    before = len(df)
    df = df[df["is_english"]].reset_index(drop=True)
    after = len(df)
    print(f" Kept {after} / {before} papers as English")

    
    print("Step 4: sentence splitting + token/lemma processing...")
    sentences_list = []
    processed_texts = []
    num_tokens = []

    for text in df["raw_text_clean"]:
        sents = get_sentences(text)
        result = preprocess_text(text)  
        processed_texts.append(result["processed_text"])
        num_tokens.append(len(result["tokens"]))
        sentences_list.append(sents)

    df["sentences"] = sentences_list      
    df["text_clean"] = processed_texts    
    df["num_tokens"] = num_tokens         
    return df

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".")) 

from src.preprocessing.preprocess import preprocess_text, clean_text, remove_special_chars

print("Imported preprocess_text, clean_text, remove_special_chars ")

Imported preprocess_text, clean_text, remove_special_chars 


In [24]:
from pathlib import Path
import pandas as pd

RAW = Path("data/raw/complete_dataset.parquet")
print("Loading:", RAW)

df_raw = pd.read_parquet(RAW)
print("Rows loaded:", len(df_raw))
print("\nColumns:\n", df_raw.columns.tolist())

Loading: data/raw/complete_dataset.parquet
Rows loaded: 1101

Columns:
 ['paper_id', 'title', 'authors', 'abstract', 'venue', 'year', 'categories', 'source', 'title_length', 'abstract_length', 'num_authors']


In [ ]:
import spacy
from langdetect import detect, LangDetectException
import unicodedata
import time

# spaCy just for sentence splitting
nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])
nlp.max_length = 2_000_000  # handle long S2ORC docs


def get_raw_text(row, full_col="full_text", abstract_col="abstract"):
    """
    Prefer full text if available, otherwise abstract.
    Change column names here if your dataset uses different ones.
    """
    full = row.get(full_col)
    abstract = row.get(abstract_col)
    if isinstance(full, str) and full.strip():
        return full
    if isinstance(abstract, str) and abstract.strip():
        return abstract
    return ""


def normalize_unicode(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return unicodedata.normalize("NFKC", text)


def is_english(text: str, min_chars: int = 200) -> bool:
    """
    Detect if text is English using langdetect on a small sample.
    """
    if not isinstance(text, str) or not text.strip():
        return False
    sample = text[: max(min_chars, 50)]
    try:
        return detect(sample) == "en"
    except LangDetectException:
        return False


def get_sentences(text: str):
    """
    Split text into sentences using spaCy, for summarization later.
    """
    if not text:
        return []
    doc = nlp(text)
    return [s.text.strip() for s in doc.sents if s.text.strip()]


def run_full_preprocessing_pipeline(
    df: pd.DataFrame,
    full_col: str = "full_text",
    abstract_col: str = "abstract",
) -> pd.DataFrame:
    """
    Uses your teammate's preprocess_text() plus extra steps:

    - build raw_text from full + abstract
    - unicode normalization
    - regex cleaning (clean_text + remove_special_chars)
    - language detection (drop non-English)
    - sentence splitting
    - tokenization/lemmatization via preprocess_text
    - num_tokens (benchmark)
    """
    df = df.copy()

    
    print("Step 1: create raw_text from full + abstract...")
    df["raw_text"] = df.apply(
        lambda row: get_raw_text(row, full_col=full_col, abstract_col=abstract_col),
        axis=1,
    )

   
    print("Step 2: unicode normalization + regex cleaning...")
    df["raw_text_clean"] = (
        df["raw_text"]
        .astype(str)
        .apply(normalize_unicode)
        .apply(clean_text)         
        .apply(remove_special_chars)  
    )

    print("Step 3: language detection (English only)...")
    df["is_english"] = df["raw_text_clean"].apply(is_english)
    before = len(df)
    df = df[df["is_english"]].reset_index(drop=True)
    after = len(df)
    print(f" Kept {after} / {before} papers as English")

    print("Step 4: sentence splitting + token/lemma processing...")
    sentences_list = []
    processed_texts = []
    num_tokens = []

    for text in df["raw_text_clean"]:
        sents = get_sentences(text)
        result = preprocess_text(text) 
        processed_texts.append(result["processed_text"])
        num_tokens.append(len(result["tokens"]))
        sentences_list.append(sents)

    df["sentences"] = sentences_list      
    df["text_clean"] = processed_texts   
    df["num_tokens"] = num_tokens        

    return df

In [25]:
df_clean = run_full_preprocessing_pipeline(df_raw,
                                           full_col="body_text",
                                           abstract_col="abstract")

Step 1: create raw_text from full + abstract...
Step 2: unicode normalization + regex cleaning...
Step 3: language detection (English only)...
 Kept 1101 / 1101 papers as English
Step 4: sentence splitting + token/lemma processing...


In [26]:

FULL_COL = "full_text"
ABSTRACT_COL = "abstract"

start = time.time()

df_clean = run_full_preprocessing_pipeline(
    df_raw,
    full_col=FULL_COL,
    abstract_col=ABSTRACT_COL,
)

elapsed = time.time() - start

print("\n⏱ Total time:", round(elapsed, 2), "seconds")
print("⏱ Avg per paper:", round(elapsed / max(len(df_clean), 1), 4), "seconds")

print("\n📊 Tokens per paper (num_tokens):")
print(df_clean["num_tokens"].describe())

Step 1: create raw_text from full + abstract...
Step 2: unicode normalization + regex cleaning...
Step 3: language detection (English only)...
 Kept 1101 / 1101 papers as English
Step 4: sentence splitting + token/lemma processing...

⏱ Total time: 123.1 seconds
⏱ Avg per paper: 0.1118 seconds

📊 Tokens per paper (num_tokens):
count    1101.000000
mean      150.742961
std        41.176717
min        20.000000
25%       124.000000
50%       150.000000
75%       177.000000
max       466.000000
Name: num_tokens, dtype: float64


In [ ]:
OUT_DIR = Path("data/preprocessed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT = OUT_DIR / "cleaned_papers.parquet"
df_clean.to_parquet(OUT, index=False)

print("💾 Saved cleaned data to:", OUT)

💾 Saved cleaned data to: data/preprocessed/cleaned_papers.parquet


In [29]:
!find . -name "*.ipynb"